### Домашнее задание

1. взять любой набор данных для бинарной классификации (можно скачать один из модельных с https://archive.ics.uci.edu/ml/datasets.php)
3. сделать feature engineering
4. обучить любой классификатор (какой вам нравится)
5. далее разделить ваш набор данных на два множества: P (positives) и U (unlabeled). Причем брать нужно не все положительные (класс 1) примеры, а только лишь часть
6. применить random negative sampling для построения классификатора в новых условиях
7. сравнить качество с решением из пункта 4 (построить отчет - таблицу метрик)
8. поэкспериментировать с долей P на шаге 5 (как будет меняться качество модели при уменьшении/увеличении размера P)

In [1]:
from __future__ import division, print_function
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pylab as plt
from sklearn.datasets import make_blobs
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_curve

In [2]:
models_results = {'Approach':[], 'precision':[],'recall':[],'f1':[], 'roc_auc':[]} 

Season in which the analysis was performed. 1) winter, 2) spring, 3) Summer, 4) fall. (-1, -0.33, 0.33, 1) 

1 Age at the time of analysis. 18-36 (0, 1) 

2 Childish diseases (ie , chicken pox, measles, mumps, polio) 1) yes, 2) no. (0, 1) 

3 Accident or serious trauma 1) yes, 2) no. (0, 1) 

4 Surgical intervention 1) yes, 2) no. (0, 1) 

5. - High fevers in the last year 1) less than three months ago, 2) more than three months ago, 3) no. (-1, 0, 1) 

6. Frequency of alcohol consumption 1) several times a day, 2) every day, 3) several times a week, 4) once a week, 5) hardly ever or never (0, 1) 

7 =Smoking habit 1) never, 2) occasional 3) daily. (-1, 0, 1) 

8 Number of hours spent sitting per day ene-16 (0, 1) 

9 Output: Diagnosis normal (N), altered (O) 



In [3]:
import pandas as pd
import numpy as np
data = pd.read_csv("fertility_Diagnosis.txt", header=None)
data.head(15)

,0,1,2,3,4,5,6,7,8,9
0,-0.33,0.69,0,1,1,0,0.8,0,0.88,N
1,-0.33,0.94,1,0,1,0,0.8,1,0.31,O
2,-0.33,0.50,1,0,0,0,1.0,-1,0.50,N
3,-0.33,0.75,0,1,1,0,1.0,-1,0.38,N
4,-0.33,0.67,1,1,0,0,0.8,-1,0.50,O
5,-0.33,0.67,1,0,1,0,0.8,0,0.50,N
6,-0.33,0.67,0,0,0,-1,0.8,-1,0.44,N
7,-0.33,1.00,1,1,1,0,0.6,-1,0.38,N
8,1.00,0.64,0,0,1,0,0.8,-1,0.25,N
9,1.00,0.61,1,0,0,0,1.0,-1,0.25,N


In [4]:
data.replace(
    {9:
     {'N': 0},
    },
    inplace=True)
data.replace(
    {9:
     {'O':1},
    },
    inplace=True)

In [5]:
data[9].value_counts()

0    88
1    12
Name: 9, dtype: int64

у нас 8 признаков и 1 целевая переменная  - бинарная 

In [6]:
print(data.shape)

(100, 10)


Всего 100 пациентов

In [7]:
from sklearn.model_selection import train_test_split

X = data.iloc[:,:-1]
y = data.iloc[:,-1]
X_train, x_test, Y_train, y_test = train_test_split(X,y, test_size=0.2, random_state=7)

In [8]:
import catboost as ctb

#model = ctb.CatBoostClassifier(iterations=100,\
#                                     max_depth=5, silent = True, random_state=21,\
#                                     eval_metric = 'F1')#,\
#                                     #reg_lambda=2)
model = RandomForestClassifier(max_depth=7, n_estimators=400)

model.fit(X_train, Y_train)
y_predict = model.predict(x_test)

In [9]:
from sklearn.metrics import recall_score, precision_score, roc_auc_score, accuracy_score, f1_score

def evaluate_results(y_test, y_predict):
    print('Classification results:')
    f1 = f1_score(y_test, y_predict)
    print("f1: %.2f%%" % (f1 * 100.0)) 
    roc = roc_auc_score(y_test, y_predict)
    print("roc: %.2f%%" % (roc * 100.0)) 
    rec = recall_score(y_test, y_predict, average='binary')
    print("recall: %.2f%%" % (rec * 100.0)) 
    prc = precision_score(y_test, y_predict, average='binary')
    print("precision: %.2f%%" % (prc * 100.0)) 
    return prc*100, rec*100, roc*100, f1*100


prc_1, rec_1, roc_1, f1_1 = evaluate_results(y_test, y_predict)

Classification results:
f1: 40.00%
roc: 69.44%
recall: 50.00%
precision: 33.33%


In [10]:
models_results['Approach'].append('Begining')
models_results['precision'].append(prc_1)
models_results['recall'].append(rec_1)
models_results['f1'].append(f1_1)
models_results['roc_auc'].append(roc_1)

### Теперь очередь за PU learning

Представим, что нам неизвестны негативы и часть позитивов

In [29]:
mod_data = data.copy()
#get the indices of the positives samples
pos_ind = np.where(mod_data.iloc[:,-1].values == 1)[0]
#shuffle them
np.random.shuffle(pos_ind)
# доля объектов, которые мы оставляем размеченными
n=0.2
pos_sample_len = int(np.ceil(n * len(pos_ind)))
print(f'Using {pos_sample_len}/{len(pos_ind)} as positives and unlabeling the rest')
pos_sample = pos_ind[:pos_sample_len]

Using 3/12 as positives and unlabeling the rest


Создаем столбец для новой целевой переменной, где у нас два класса - P (1) и U (-1)

In [30]:
mod_data['class_test'] = -1
mod_data.loc[pos_sample,'class_test'] = 1
print('target variable:\n', mod_data.iloc[:,-1].value_counts())

target variable:
 -1    97
 1     3
Name: class_test, dtype: int64


* We now have just 4 positive samples labeled as 1 in the 'class_test' col while the rest is unlabeled as -1.

In [31]:
mod_data.head(30)

,0,1,2,3,4,5,6,7,8,9,class_test
0,-0.33,0.69,0,1,1,0,0.8,0,0.88,0,-1
1,-0.33,0.94,1,0,1,0,0.8,1,0.31,1,1
2,-0.33,0.50,1,0,0,0,1.0,-1,0.50,0,-1
3,-0.33,0.75,0,1,1,0,1.0,-1,0.38,0,-1
4,-0.33,0.67,1,1,0,0,0.8,-1,0.50,1,-1
5,-0.33,0.67,1,0,1,0,0.8,0,0.50,0,-1
6,-0.33,0.67,0,0,0,-1,0.8,-1,0.44,0,-1
7,-0.33,1.00,1,1,1,0,0.6,-1,0.38,0,-1
8,1.00,0.64,0,0,1,0,0.8,-1,0.25,0,-1
9,1.00,0.61,1,0,0,0,1.0,-1,0.25,0,-1


Remember that this data frame (x_data) includes the former target variable that we keep here just to compare the results

[:-2] is the original class label for positive and negative data [:-1] is the new class for positive and unlabeled data

In [32]:
mod_data = mod_data.sample(frac=1)
neg_sample = mod_data[mod_data['class_test']==-1][:len(mod_data[mod_data['class_test']==1])]
sample_test = mod_data[mod_data['class_test']==-1][len(mod_data[mod_data['class_test']==1]):]
pos_sample = mod_data[mod_data['class_test']==1]
print(neg_sample.shape, pos_sample.shape)
sample_train = pd.concat([neg_sample, pos_sample]).sample(frac=1)

(3, 11) (3, 11)


In [33]:
x_data = mod_data.iloc[:,:-2].values # just the X 
y_labeled = mod_data.iloc[:,-1].values # new class (just the P & U)
y_positive = mod_data.iloc[:,-2].values # original class

In [34]:
model = RandomForestClassifier()

model.fit(sample_train.iloc[:,:-2].values, 
          sample_train.iloc[:,-2].values)
y_predict = model.predict(sample_test.iloc[:,:-2].values)


In [35]:
prc_2, rec_2, f1_2, roc_2 = evaluate_results(sample_test.iloc[:,-2].values, y_predict)

Classification results:
f1: 19.35%
roc: 55.69%
recall: 66.67%
precision: 11.32%


In [36]:
models_results['Approach'].append(f'random negative sampling n = {n}')
models_results['precision'].append(prc_2)
models_results['recall'].append(rec_2)
models_results['f1'].append(f1_2)
models_results['roc_auc'].append(roc_2)

In [37]:
print(pd.DataFrame(models_results))

                           Approach  precision     recall         f1  \
0                          Begining  33.333333  50.000000  40.000000   
1  random negative sampling n = 0.5   8.333333  50.000000  54.878049   
2  random negative sampling n = 0.3  13.043478  75.000000  63.690476   
3  random negative sampling n = 0.2  11.320755  66.666667  55.686275   

     roc_auc  
0  69.444444  
1  14.285714  
2  22.222222  
3  19.354839  


<b>Бонусный вопрос:</b>

Как вы думаете, какой из методов на практике является более предпочтительным: random negative sampling или 2-step approach?

Ваш ответ здесь:

2-step approach
